<a href="https://colab.research.google.com/github/chaudharynidhi/Image_Classification/blob/master/Intel_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
import tensorflow as tf
from tensorflow import keras
import os
import PIL
import PIL.Image
import pathlib
import shutil

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -uq "/content/drive/MyDrive/data_intel/archive.zip" -d "/content/drive/MyDrive/data_intel/"

KeyboardInterrupt: ignored

In [6]:
train_data = "/content/drive/MyDrive/data_intel/seg_train/seg_train"
test_data = "/content/drive/MyDrive/data_intel/seg_test/seg_test"
pred_data = "/content/drive/MyDrive/data_intel/seg_pred/seg_pred"

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2, zoom_range=0.2, 
                                   horizontal_flip=True)

train_set = train_datagen.flow_from_directory(train_data, target_size = (224,224), 
                                              batch_size=4, class_mode = 'sparse')

Found 14034 images belonging to 6 classes.


In [8]:
import tensorflow.keras.preprocessing.image

In [9]:
test_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2, zoom_range=0.2, 
                                  horizontal_flip=True)

test_set = test_datagen.flow_from_directory(test_data, target_size = (224,224), 
                                            batch_size=4, class_mode = 'sparse')

Found 3000 images belonging to 6 classes.


In [10]:
len(test_set.class_indices)

6

In [11]:
batch_size=4


In [12]:
final_image = tf.keras.applications.xception.preprocess_input
final_image

<function tensorflow.python.keras.applications.xception.preprocess_input>

In [13]:

base_model = tf.keras.applications.Xception(input_shape=(224,224,3), include_top=False, 
                                            classifier_activation='relu', classes=len(train_set.class_indices))

83689472/83683744 [==============================] - 1s 0us/step


In [14]:
image_batch, label_batch = next(iter(train_set))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(4, 7, 7, 2048)


In [15]:
base_model.trainable=False

In [16]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(4, 2048)


In [17]:
prediction_layer = tf.keras.layers.Dense(6, activation='sigmoid')
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(4, 6)


In [18]:
inputs = tf.keras.Input(shape=(224, 224, 3))
x = final_image(inputs)
x = base_model(x, training = False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 224, 224, 3)       0         
_________________________________________________________________
tf.math.subtract (TFOpLambda (None, 224, 224, 3)       0         
_________________________________________________________________
xception (Functional)        (None, 7, 7, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 12294 

In [19]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [20]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 224, 224, 3)       0         
_________________________________________________________________
tf.math.subtract (TFOpLambda (None, 224, 224, 3)       0         
_________________________________________________________________
xception (Functional)        (None, 7, 7, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 12294 

In [ ]:
history = model.fit(train_set,
                    epochs=20,
                    validation_data=test_set)

Epoch 1/20
1726/3509 [=============>................] - ETA: 29:53 - loss: 1.7897 - accuracy: 0.1928

In [ ]:
model.trainable_variables

In [ ]:
model.evaluate(test_set)